In [1]:
import os 

from langchain.agents import create_csv_agent
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings


In [2]:
from langchain.document_loaders.csv_loader import CSVLoader

In [5]:
import sys
import csv

csv.field_size_limit(sys.maxsize)

loader = CSVLoader(file_path='data/YouTube_transcripts_Kaggle.csv', 
                  csv_args={
                      "delimiter":",",
                      # "fieldnames":["title", "author", "transcript", "playlist_name"],
                      },
                   # source_column="transcript"
                  )
data = loader.load()

In [6]:
data[2].page_content

"title: Hyperparameter Tuning on Vertex AI\nauthor: Google Cloud Tech\ntranscript: PRIYANKA VERGADIA: Hi. I'm Priyanka Vergadia. And in this video, we will\nsee what are hyperparameters, how to tune them, and set\nup a hyperparameter tuning job in Vertex AI. All right, so what\nis a hyperparameter? Well, a training application\nhandles three categories of data as it trains the model. The input data, which is\nalso called training data, contains the features that the\nmodel uses to make predictions. Now, but this data\nis never directly part of the model\narchitecture itself. The second type of data\nis model parameters. These are the variables that\nour ML model or technique uses to adjust the data. For example, a deep\nneural network, or DNN, is composed of processing\nnodes or neurons, each with an operation\nperformed on the data as it travels\nthrough the network. Now, when our DNN is\ntrained, each node has a weight value that tells\nour model how much impact it has on the final p

In [8]:
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
)
from langchain.vectorstores import Chroma


In [9]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
pages = text_splitter.split_text(data[0].page_content)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.create_documents(pages)

# print(texts)
len(texts)

# dataset_path = "hub://" + org + "/data"
# embeddings = OpenAIEmbeddings()
# db = DeepLake.from_documents(
#     texts, embeddings, dataset_path=dataset_path, overwrite=True
# )

7

In [9]:
texts[1].metadata

{}

In [26]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-ada-002")
vectorstore = Chroma.from_documents(texts,embedding=embeddings, persist_directory="./chroma_db")

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..


In [6]:
# import
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

# load the document and split it into chunks

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma

In [10]:
db = Chroma.from_documents(texts, embedding_function)

In [27]:
# query it
query = "how to use feature store"
docs = vectorstore.similarity_search_with_score(query)

# print results
# print(docs[0].page_content)

In [28]:
docs

[(Document(page_content="quality to degrade over time. That is exactly where Vertex\nFeature Store comes in. It's a fully managed\nand unified solution to share, discover,\nand serve machine learning features at scale\nacross different teams within your organization. And it also helps\nreduce the time to build and deploy\nyour AI ML applications by making it easy to\nmanage and organize your ML features in one place. It makes the features reusable,\neasy to serve, and avoids skew. Now let's see how to set it up. In the console, in Vertex\nAI, we see the Feature tab. To get started, let's\nclick on this documentation and explore, using\nFeature Store section. Now the first thing you\nneed is a Feature Store. At the time of this\nrecording, Feature Store is in preview so just\nknow that depending on when you're watching this,\nthere might be more options and updates that you would see. You cannot create a Feature\nStore in the console, so let's use this sample", metadata={}),
  0.2988402

In [35]:
from langchain.chains import RetrievalQA
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma


# db = Chroma.from_documents(texts, embeddings)
# expose this index in a retriever interface
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":2})

# create a chain to answer questions 
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(openai_api_key=OPENAI_API_KEY), chain_type="stuff", retriever=retriever, return_source_documents=True, verbose=True)

In [31]:
query = "How to use the feature store?"
result = qa({"query": query})

In [32]:
result

{'query': 'How to use the feature store?',
 'result': ' To use the feature store, you need to create a Feature Store using the SDK. You can follow the instructions in the sample notebook which shows how to import features into the Feature Store, serve online prediction requests using the imported features, and access imported features in offline jobs such as training jobs. The Feature Store organizes data with three hierarchical concepts: Feature Store, Feature Group, and Feature.',
 'source_documents': [Document(page_content="quality to degrade over time. That is exactly where Vertex\nFeature Store comes in. It's a fully managed\nand unified solution to share, discover,\nand serve machine learning features at scale\nacross different teams within your organization. And it also helps\nreduce the time to build and deploy\nyour AI ML applications by making it easy to\nmanage and organize your ML features in one place. It makes the features reusable,\neasy to serve, and avoids skew. Now le

In [43]:
query = "How to use neural networks to train Alexnet?"
result_02 = qa({"query": query})




> Entering new  chain...

> Finished chain.


In [41]:
def print_res(res):
    print(res['result'])
    print("Docs::")
    for i, doc in enumerate(res["source_documents"]):
        print(f'{i}')
        print(doc.page_content)

In [44]:
print_res(result_02)

 I don't know.
Docs::
0
Store in the console, so let's use this sample
notebook to learn how to create it using the SDK. This sample uses a movie
recommendations data set and the task
is to train a model to predict if a user is
going to watch a movie and serve this model online. We will learn to import our
features into Feature Store, serve online prediction requests
using the imported features, and then access imported
features in offline jobs, such as training jobs. To set up, we install
some additional packages, set up our project, and
authenticate our Google Cloud account. Step one is to create
data set for output. We are creating BigQuery data
set to host the output data. Input the name of the
data set and the table we want to store
the output later. Then we are defining
constants and Feature Store-related imports. Here's how the Vertex
Feature Store actually works. It organizes the data with the
three hierarchical concepts. Featurestore, which is the
1
title: Introduction to Vert

In [45]:
result_02

{'query': 'How to use neural networks to train Alexnet?',
 'result': " I don't know.",
 'source_documents': [Document(page_content="Store in the console, so let's use this sample\nnotebook to learn how to create it using the SDK. This sample uses a movie\nrecommendations data set and the task\nis to train a model to predict if a user is\ngoing to watch a movie and serve this model online. We will learn to import our\nfeatures into Feature Store, serve online prediction requests\nusing the imported features, and then access imported\nfeatures in offline jobs, such as training jobs. To set up, we install\nsome additional packages, set up our project, and\nauthenticate our Google Cloud account. Step one is to create\ndata set for output. We are creating BigQuery data\nset to host the output data. Input the name of the\ndata set and the table we want to store\nthe output later. Then we are defining\nconstants and Feature Store-related imports. Here's how the Vertex\nFeature Store actually 